# **Desenvolvimento de um Sistema de Agregação de Dados de Mídias Sociais**

**Objectivo:** Introduzir a interação com APIs, o armazenamento em bancos de 
dados NoSQL

In [1]:
import pandas as pd
import requests
import os
from dotenv import load_dotenv
import pymongo


token = os.getenv('bear_token')
load_dotenv()
print(token)

AAAAAAAAAAAAAAAAAAAAAF0F2QEAAAAAE86URv4qyCFy3ij5DP%2B%2FcVuas4c%3Dt6pS6XqeNfEiAgVzecKGuzrHUs8g1EthcEimiUwq7xBWSzPn0O


# **Extrat(Twitter)**

In [2]:
# configurando os parametros e o cabeçalho
headers = {
    'Authorization': f'Bearer {token}'
}
params = {
    "query": "Linguagem Python",
    "max_results": 10,
    "tweet.fields": "created_at,public_metrics,author_id",
    "expansions": "author_id",
    "user.fields": "username,name"
}

url = "https://api.twitter.com/2/tweets/search/recent"
response = requests.get(url, headers=headers, params=params)
print(response.status_code)
response = response.json()
print(response)

429
{'title': 'Too Many Requests', 'detail': 'Too Many Requests', 'type': 'about:blank', 'status': 429}


In [3]:
response.keys()

dict_keys(['title', 'detail', 'type', 'status'])

In [4]:
type(response['data'])

KeyError: 'data'

# **Transform(Twitter)**

In [6]:
autores = response['includes']['users']
autores_df = pd.DataFrame(autores)
tweets = response['data']
tweets_df = pd.DataFrame(tweets)
tweets_df = tweets_df.merge(autores_df, left_on='author_id', right_on='id', how='left')
tweets_df = tweets_df[['created_at', 'text', 'public_metrics', 'username', 'name']]
tweets_df['retweet_count'] = tweets_df['public_metrics'].apply(lambda x: x['retweet_count'])
tweets_df['reply_count'] = tweets_df['public_metrics'].apply(lambda x: x['reply_count'])
tweets_df['like_count'] = tweets_df['public_metrics'].apply(lambda x: x['like_count'])
tweets_df['quote_count'] = tweets_df['public_metrics'].apply(lambda x: x['quote_count'])
tweets_df = tweets_df.drop(columns=['public_metrics'])
tweets_df['created_at'] = pd.to_datetime(tweets_df['created_at'])
tweets_df = tweets_df.sort_values(by='created_at', ascending=False)
tweets_df = tweets_df.reset_index(drop=True)
tweets_df.head()

KeyError: 'includes'

In [ ]:
response['data']

[{'author_id': '1264606443468140545',
  'public_metrics': {'retweet_count': 0,
   'reply_count': 1,
   'like_count': 0,
   'quote_count': 0,
   'bookmark_count': 0,
   'impression_count': 38},
  'text': 'Ainda não tanko que aquele cara meteu um "é tipo um aluno de biologia não saber uma célula" porque eu "não sabia" python, sendo que qualquer linguagem de programação é claramente um condicionamento e as células na computação são os componentes de um computador',
  'created_at': '2025-06-11T19:34:41.000Z',
  'edit_history_tweet_ids': ['1932884210127388818'],
  'id': '1932884210127388818'},
 {'author_id': '1160702238043574275',
  'public_metrics': {'retweet_count': 0,
   'reply_count': 0,
   'like_count': 0,
   'quote_count': 0,
   'bookmark_count': 0,
   'impression_count': 21},
  'text': 'engolindo meu preconceito, python realmente é uma linguagem muito foda',
  'created_at': '2025-06-11T18:58:51.000Z',
  'edit_history_tweet_ids': ['1932875193526952285'],
  'id': '1932875193526952285'}

In [ ]:
response['includes']['users']

[{'id': '1264606443468140545', 'name': 'Sean.py 🔞', 'username': 'misosccp'},
 {'id': '1160702238043574275', 'name': 'Talis ᶜʳᶠ', 'username': 'blackttpx'},
 {'id': '1371141048286732290',
  'name': '🌻 Floralzito 🌻',
  'username': 'Floral_Ghost0'},
 {'id': '1299498504675627009',
  'name': 'Limeinhalf – nns/acc',
  'username': 'limeinhalf'},
 {'id': '1601923271880019969',
  'name': 'Igor.js',
  'username': 'Igaosacanagens'},
 {'id': '1278740349566017543', 'name': 'ؘ', 'username': 'jinyfilms'},
 {'id': '1755385867554680832',
  'name': '自虐Arvesiak✨🩸',
  'username': 'Cannibalcel_'},
 {'id': '1421815220213796884', 'name': '#bolhadev', 'username': 'BolhaDevs'},
 {'id': '1590786931453034501', 'name': 'Gautcho', 'username': 'devgaucho'},
 {'id': '1274055423176462339', 'name': 'ॐ', 'username': 'alftersex'}]

In [ ]:
name = [autores['name'] for autores in autores]
username = [autores['username'] for autores in autores]

In [ ]:
data_transform = []
dados = response['data']
show_index = 0
for data in dados:
    id = data['id']
    texto = data['text']
    numero_retweets = data['public_metrics']['retweet_count']
    numero_likes = data['public_metrics']['like_count']
    criado_em = data['created_at']
    ID_autor = data['author_id']
    data_transform.append(
        {
            # 'ID': id,
            'Nome': name[show_index],
            'Username': username[show_index],
            'Texto': texto,
            'Numero_Retweets': numero_retweets,
            'Numero_likes': numero_likes,
            'Data da criação': criado_em
            # 'ID_author': ID_autor
        }
    )

    show_index += 1
data_transform

[{'Nome': 'Sean.py 🔞',
  'Username': 'misosccp',
  'Texto': 'Ainda não tanko que aquele cara meteu um "é tipo um aluno de biologia não saber uma célula" porque eu "não sabia" python, sendo que qualquer linguagem de programação é claramente um condicionamento e as células na computação são os componentes de um computador',
  'Numero_Retweets': 0,
  'Numero_likes': 0,
  'Data da criação': '2025-06-11T19:34:41.000Z'},
 {'Nome': 'Talis ᶜʳᶠ',
  'Username': 'blackttpx',
  'Texto': 'engolindo meu preconceito, python realmente é uma linguagem muito foda',
  'Numero_Retweets': 0,
  'Numero_likes': 0,
  'Data da criação': '2025-06-11T18:58:51.000Z'},
 {'Nome': '🌻 Floralzito 🌻',
  'Username': 'Floral_Ghost0',
  'Texto': 'Eu vi um querido com o uniforme do curso dele hoje e tinha o símbolo do python na camisa, eu achei tão fofinho isso... Já pensou se fosse tipo as casas de Hogwarts e todo mundo tivesse que usar a farda com símbolo da linguagem de programação que se especializou?!',
  'Numero_Retw

In [ ]:
df = pd.DataFrame(data_transform)
df

,Nome,Username,Texto,Numero_Retweets,Numero_likes,Data da criação
0,Sean.py 🔞,misosccp,"Ainda não tanko que aquele cara meteu um ""é ti...",0,0,2025-06-11T19:34:41.000Z
1,Talis ᶜʳᶠ,blackttpx,"engolindo meu preconceito, python realmente é ...",0,0,2025-06-11T18:58:51.000Z
2,🌻 Floralzito 🌻,Floral_Ghost0,Eu vi um querido com o uniforme do curso dele ...,0,2,2025-06-11T18:23:42.000Z
3,Limeinhalf – nns/acc,limeinhalf,@W1N_Nerd @marcioportes Não estou convencido q...,0,2,2025-06-11T18:00:08.000Z
4,Igor.js,Igaosacanagens,- To estudando programação haha logo to trabal...,0,0,2025-06-11T12:02:42.000Z
5,ؘ,jinyfilms,meu lado ahgase é quando vejo python é imagina...,0,1,2025-06-11T02:19:11.000Z
6,自虐Arvesiak✨🩸,Cannibalcel_,"@nuntinteligente Eu nem sei, não fiz nada, só ...",0,0,2025-06-11T00:10:46.000Z
7,#bolhadev,BolhaDevs,RT @devgaucho: o pessoal confunde muito framew...,1,0,2025-06-10T21:08:20.000Z
8,Gautcho,devgaucho,o pessoal confunde muito framework com linguag...,1,1,2025-06-10T21:05:37.000Z
9,ॐ,alftersex,"o negócio começou em C++, mas ontem fizemos em...",0,0,2025-06-10T21:01:19.000Z
